In [1]:
import tensorflow as tf

In [32]:
!pip install sklearn
!pip install matplotlib
!pip install transformers
!pip install tqdm
!pip install pandas
!pip install numpy
!pip install torch

In [49]:
!pip install ipywidgets

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1080'

In [4]:
df = pd.read_csv("./IMDB Dataset.csv", delimiter=',')

In [5]:
df.shape

(49999, 2)

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
sentences = df.review.values

In [8]:
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]

In [9]:
labels = df.sentiment.values
labels = [1 if label == 'positive' else 0 for label in labels]

In [10]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [11]:
tokenized_texts = [tokenizer.tokenize(sentence) for sentence in sentences]

In [12]:
print (tokenized_texts[0])

['▁one', '▁of', '▁the', '▁other', '▁reviewer', 's', '▁has', '▁mentioned', '▁that', '▁after', '▁watching', '▁just', '▁1', '▁', 'oz', '▁episode', '▁you', "'", 'll', '▁be', '▁hooked', '.', '▁they', '▁are', '▁right', ',', '▁as', '▁this', '▁is', '▁exactly', '▁what', '▁happened', '▁with', '▁me', '.', '<', 'br', '▁', '/', '>', '<', 'br', '▁', '/', '>', 'the', '▁first', '▁thing', '▁that', '▁struck', '▁me', '▁about', '▁', 'oz', '▁was', '▁its', '▁brutality', '▁and', '▁un', 'fli', 'nch', 'ing', '▁scenes', '▁of', '▁violence', ',', '▁which', '▁set', '▁in', '▁right', '▁from', '▁the', '▁word', '▁go', '.', '▁trust', '▁me', ',', '▁this', '▁is', '▁not', '▁a', '▁show', '▁for', '▁the', '▁faint', '▁', 'hearted', '▁or', '▁timid', '.', '▁this', '▁show', '▁pulls', '▁no', '▁punches', '▁with', '▁regards', '▁to', '▁drugs', ',', '▁sex', '▁or', '▁violence', '.', '▁its', '▁is', '▁hardcore', ',', '▁in', '▁the', '▁classic', '▁use', '▁of', '▁the', '▁word', '.', '<', 'br', '▁', '/', '>', '<', 'br', '▁', '/', '>', 'it',

In [13]:
MAX_LEN = 128

In [14]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [15]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [16]:
input_ids

array([[  65,   20,   18, ...,  167, 3151,  669],
       [  24, 3239,  293, ..., 1062,   21, 2062],
       [  17,  150,  449, ...,   65,   20, 3302],
       ...,
       [  17,  150,  569, ...,   27,  116, 5182],
       [  17,  150,   26, ...,   19,   33,  106],
       [ 116,   65, 5883, ...,   74,  248,   52]])

In [17]:
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [18]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=1029, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=1029, test_size=0.2)

In [20]:
# split the data set into training, validation and test set
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [21]:
batch_size = 4

In [22]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [23]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [24]:
model.cuda()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [25]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [26]:
# This variable contains all of the hyperparemeter information and the learning rate.
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [27]:
train_loss_set = []

epochs = 4

for _ in trange(epochs, desc="Epoch"):
  # Set our model to training mode (as opposed to evaluation mode)
    model.train()
  
  # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        train_loss_set.append(loss.item())    

        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
    
    
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:  25%|███████████████████████████████████████                                                                                                                     | 1/4 [22:38<1:07:55, 1358.36s/it]

Train loss: 0.31759870747576935


Epoch:  50%|███████████████████████████████████████████████████████████████████████████████                                                                               | 2/4 [45:23<45:24, 1362.46s/it]

Train loss: 0.220070587658172


Epoch:  75%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                       | 3/4 [1:07:43<22:32, 1352.32s/it]

Train loss: 0.15134536687863875


Epoch: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [1:29:58<00:00, 1349.52s/it]

Train loss: 0.10762934819170332


In [30]:
torch.save(model.state_dict(), './xlnet_model_1029.ckpt')

In [32]:
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(validation_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        # print (outputs)
        prediction = torch.argmax(outputs[0],dim=1)
        total += b_labels.size(0)
        correct+=(prediction==b_labels).sum().item()

In [33]:
print('Test Accuracy of the model on vla data is: {} %'.format(100 * correct / total))

Test Accuracy of the model on vla data is: 89.2 %
